## 1. load data files & check

In [116]:
import pandas as pd
df_train = pd.read_csv('./titanic/train.csv')
df_test = pd.read_csv("./titanic/test.csv")
#print(type(df_train))

In [117]:
# check train data
print(df_train.head())
print(df_train.shape)
df_train.info()
df_train.isnull().sum()

PassengerId  Survived  Pclass  \
0            1         0       3   
1            2         1       1   
2            3         1       3   
3            4         1       1   
4            5         0       3   

                                                Name     Sex   Age  SibSp  \
0                            Braund, Mr. Owen Harris    male  22.0      1   
1  Cumings, Mrs. John Bradley (Florence Briggs Th...  female  38.0      1   
2                             Heikkinen, Miss. Laina  female  26.0      0   
3       Futrelle, Mrs. Jacques Heath (Lily May Peel)  female  35.0      1   
4                           Allen, Mr. William Henry    male  35.0      0   

   Parch            Ticket     Fare Cabin Embarked  
0      0         A/5 21171   7.2500   NaN        S  
1      0          PC 17599  71.2833   C85        C  
2      0  STON/O2. 3101282   7.9250   NaN        S  
3      0            113803  53.1000  C123        S  
4      0            373450   8.0500   NaN        S
(891, 1

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [118]:
# check test data
print(df_test.head())
print(df_test.shape)
df_test.head()
df_test.info()
df_test.isnull().sum()

PassengerId  Pclass                                          Name     Sex  \
0          892       3                              Kelly, Mr. James    male   
1          893       3              Wilkes, Mrs. James (Ellen Needs)  female   
2          894       2                     Myles, Mr. Thomas Francis    male   
3          895       3                              Wirz, Mr. Albert    male   
4          896       3  Hirvonen, Mrs. Alexander (Helga E Lindqvist)  female   

    Age  SibSp  Parch   Ticket     Fare Cabin Embarked  
0  34.5      0      0   330911   7.8292   NaN        Q  
1  47.0      1      0   363272   7.0000   NaN        S  
2  62.0      0      0   240276   9.6875   NaN        Q  
3  27.0      0      0   315154   8.6625   NaN        S  
4  22.0      1      1  3101298  12.2875   NaN        S
(418, 11)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 418 entries, 0 to 417
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       -------

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

## 1. Feature Engineering
### get 'title' from names

In [119]:
train_test_data = [df_train, df_test] # combining train and test dataset

for dataset in train_test_data:
    dataset['Title'] = dataset['Name'].str.extract(' ([A-Za-z]+)\.', expand=False)

print(df_train['Title'].value_counts())
print(df_test['Title'].value_counts())

Mr          517
Miss        182
Mrs         125
Master       40
Dr            7
Rev           6
Col           2
Mlle          2
Major         2
Capt          1
Lady          1
Ms            1
Jonkheer      1
Don           1
Sir           1
Mme           1
Countess      1
Name: Title, dtype: int64
Mr        240
Miss       78
Mrs        72
Master     21
Col         2
Rev         2
Dr          1
Dona        1
Ms          1
Name: Title, dtype: int64


In [120]:
# map data with string (not number for test autoML performance)
title_mapping = {"Mr": "Mr", "Miss": "Miss", "Mrs": "Mrs", 
                 "Master": "Gentleman", "Dr": "Gentleman", "Rev": "Gentleman", "Col": "unknown", "Major": "Gentleman", "Mlle": "unknown","Countess": "Lady",
                 "Ms": "Miss", "Lady": "Lady", "Jonkheer": "unknown", "Don": "Gentleman", "Dona" : "Lady", "Mme": "unknown","Capt": "Gentleman","Sir": "Gentleman" }
for dataset in train_test_data:
    dataset['Title'] = dataset['Title'].map(title_mapping)
print(df_train['Title'].value_counts())
print(df_test['Title'].value_counts())

Mr           517
Miss         183
Mrs          125
Gentleman     58
unknown        6
Lady           2
Name: Title, dtype: int64
Mr           240
Miss          79
Mrs           72
Gentleman     24
unknown        2
Lady           1
Name: Title, dtype: int64


### drop name 

In [121]:
# delete unnecessary feature from dataset
df_train.drop('Name', axis=1, inplace=True)
df_test.drop('Name', axis=1, inplace=True)
df_train.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr
1,2,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs
2,3,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss
3,4,1,1,female,35.0,1,0,113803,53.1000,C123,S,Mrs
4,5,0,3,male,35.0,0,0,373450,8.0500,NaN,S,Mr


### fill missed age

In [122]:
# merge for calculate overall statstics
# (pd.merge(df_train,df_test)).head()
## NEED INVESTGATE!
# df_all = (pd.concat([df_train,df_test]))
# df_all.info()

# df_all.isnull().sum()

In [123]:
# df_test.groupby("Title")["Age"].transform("median")
# # fill missing age with median age for each title (Mr, Mrs, Miss, Others)
df_train["Age"].fillna(df_train.groupby("Title")["Age"].transform("median"), inplace=True)
df_test["Age"].fillna(df_test.groupby("Title")["Age"].transform("median"), inplace=True)
df_train.head()


,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr
1,2,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs
2,3,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss
3,4,1,1,female,35.0,1,0,113803,53.1000,C123,S,Mrs
4,5,0,3,male,35.0,0,0,373450,8.0500,NaN,S,Mr


### fill embarked

In [124]:
for dataset in train_test_data:
    dataset['Embarked'] = dataset['Embarked'].fillna('S')

### fill fare

In [125]:
# fill missing Fare with median fare for each Pclass
df_train["Fare"].fillna(df_train.groupby("Pclass")["Fare"].transform("median"), inplace=True)
df_test["Fare"].fillna(df_test.groupby("Pclass")["Fare"].transform("median"), inplace=True)
df_train.head(50)

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr
1,2,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C,Mrs
2,3,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Miss
3,4,1,1,female,35.0,1,0,113803,53.1000,C123,S,Mrs
4,5,0,3,male,35.0,0,0,373450,8.0500,NaN,S,Mr
5,6,0,3,male,30.0,0,0,330877,8.4583,NaN,Q,Mr
6,7,0,1,male,54.0,0,0,17463,51.8625,E46,S,Mr
7,8,0,3,male,2.0,3,1,349909,21.0750,NaN,S,Gentleman
8,9,1,3,female,27.0,0,2,347742,11.1333,NaN,S,Mrs
9,10,1,2,female,14.0,1,0,237736,30.0708,NaN,C,Mrs


### fill cabin

In [126]:

# map to number
cabin_mapping = {"A": 0, "B": 1, "C": 2, "D": 3, "E": 4, "F": 5, "G": 6, "T": 7}
for dataset in train_test_data:
    dataset["Cabin"] = dataset["Cabin"].str[:1]
    dataset["Cabin"] = dataset["Cabin"].map(cabin_mapping)

# fill missing Fare with median fare for each Pclass
df_train["Cabin"].fillna(df_train.groupby("Pclass")["Cabin"].transform("median"), inplace=True)
df_test["Cabin"].fillna(df_test.groupby("Pclass")["Cabin"].transform("median"), inplace=True)

### simplize family

In [ ]:
df_train["FamilySize"] = df_train["SibSp"] + df_train["Parch"] + 1
df_test["FamilySize"] = df_test["SibSp"] + df_test["Parch"] + 1

In [134]:
# delete unnecessary feature from dataset
features_drop = ['PassengerId', 'Ticket', 'SibSp', 'Parch']
df_train = df_train.drop(features_drop, axis=1)
test = test.drop(features_drop, axis=1)

df_train.drop('PassengerId', axis=1, inplace=True)
df_test.drop('PassengerId', axis=1, inplace=True)
df_train.drop('Ticket', axis=1, inplace=True)
df_test.drop('Ticket', axis=1, inplace=True)
df_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Title
0,3,male,22.0,1,0,7.2500,5.0,S,Mr
1,1,female,38.0,1,0,71.2833,2.0,C,Mrs
2,3,female,26.0,0,0,7.9250,5.0,S,Miss
3,1,female,35.0,1,0,53.1000,2.0,S,Mrs
4,3,male,35.0,0,0,8.0500,5.0,S,Mr


In [128]:
import autokeras as ak
# Initialize the structured data classifier.
clf = ak.StructuredDataClassifier(max_trials=1) # It tries 1 different models.
# Feed the structured data classifier with training data.
train_y = df_train.pop('Survived')
clf.fit(x=df_train,y=train_y)

Train for 23 steps, validate for 6 steps
Epoch 1/1000
 1/23 [>.............................] - ETA: 21s - loss: 3.0434 - accuracy: 0.50012/23 [==============>...............] - ETA: 0s - loss: 1.9199 - accuracy: 0.6023/23 [==============================] - 1s 65ms/step - loss: 1.5083 - accuracy: 0.6129 - val_loss: 0.8226 - val_accuracy: 0.4719
Epoch 2/1000
 1/23 [>.............................] - ETA: 0s - loss: 1.0207 - accuracy: 0.65 8/23 [=========>....................] - ETA: 0s - loss: 0.9357 - accuracy: 0.6413/23 [===============>..............] - ETA: 0s - loss: 0.9400 - accuracy: 0.6123/23 [==============================] - 0s 9ms/step - loss: 0.8394 - accuracy: 0.6424 - val_loss: 0.7919 - val_accuracy: 0.3820
Epoch 3/1000
 1/23 [>.............................] - ETA: 0s - loss: 0.7262 - accuracy: 0.5911/23 [=============>................] - ETA: 0s - loss: 0.7178 - accuracy: 0.6419/23 [=======================>......] - ETA: 0s - loss: 0.7168 - accuracy: 0.6420/23 [============

.................] - ETA: 0s - loss: 0.6618 - accuracy: 0.6914/29 [=============>................] - ETA: 0s - loss: 0.6597 - accuracy: 0.7128/29 [===========================>..] - ETA: 0s - loss: 0.6191 - accuracy: 0.7229/29 [==============================] - 0s 9ms/step - loss: 0.6175 - accuracy: 0.7239 - val_loss: 0.4476 - val_accuracy: 0.7753
Epoch 28/1000
 1/29 [>.............................] - ETA: 0s - loss: 0.5998 - accuracy: 0.6514/29 [=============>................] - ETA: 0s - loss: 0.6638 - accuracy: 0.7029/29 [==============================] - 0s 6ms/step - loss: 0.6180 - accuracy: 0.7205 - val_loss: 0.4455 - val_accuracy: 0.7697
Epoch 29/1000
 1/29 [>.............................] - ETA: 0s - loss: 0.6007 - accuracy: 0.6516/29 [===============>..............] - ETA: 0s - loss: 0.6492 - accuracy: 0.7121/29 [====================>.........] - ETA: 0s - loss: 0.6202 - accuracy: 0.7229/29 [==============================] - 0s 8ms/step - loss: 0.6119 - accuracy: 0.7239 - val_l

In [129]:
preds = clf.predict(df_test)

In [130]:
df_pred = pd.read_csv("./titanic/gender_submission.csv")
df_pred.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [131]:
df_pred['Survived'] = preds
df_pred.head()

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1


In [132]:
df_pred.to_csv("./titanic_submission.csv", index=None)